In [ ]:
from dataclasses import asdict
import math
import random
import itertools

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from minitorch.module import LinearScalarLayer, LinearTensorLayer, ScalarNetwork, TensorNetwork
from minitorch.autodiff import Scalar
import minitorch.autodiff.tensor_functions as tf
from minitorch.optim import SGDOptimizer
from minitorch.datasets import Datasets, DatasetTypes
from minitorch.operators import sigmoid
import minitorch.scalar_metrics as sm
import minitorch.scalar_losses as sl
import minitorch.tensor_losses as tl
import minitorch.tensor_metrics as tm

In [ ]:
# Create a dataset
N_SAMPLES = 50
dataset_type = DatasetTypes.xor

datasets = Datasets.generate_datasets(n_samples=N_SAMPLES)
dataset = asdict(datasets)[dataset_type]

pos_class = [x for (x, y) in zip(dataset.xs, dataset.ys) if y == 1]
neg_class = [x for (x, y) in zip(dataset.xs, dataset.ys) if y == 0]

x1_pos, x2_pos = zip(*pos_class)
x1_neg, x2_neg = zip(*neg_class)

fig, ax = plt.subplots(1, 1, dpi=110)
ax.scatter(list(x1_pos), list(x2_pos), marker="x", c="tab:blue", label="class = 1")
ax.scatter(list(x1_neg), list(x2_neg), marker="o", c="tab:red", label="class = 0")
ax.legend(loc=1)

bottom_left = Rectangle((0, 0), 0.5, 0.5, color="tab:red", alpha=0.2, lw=0.0)
top_right = Rectangle((0.5, 0.5), 0.5, 0.5, color="tab:red", alpha=0.2, lw=0.0)
top_left = Rectangle((0.0, 0.5), 0.5, 0.5, color="tab:blue", alpha=0.2, lw=0.0)
bottom_right = Rectangle((0.5, 0.0), 0.5, 0.5, color="tab:blue", alpha=0.2, lw=0.0)
ax.add_patch(bottom_left)
ax.add_patch(bottom_right)
ax.add_patch(top_left)
ax.add_patch(top_right)

ax.set_xlabel("x1")
ax.set_ylabel("x2")

fig.tight_layout()

### Scalars

In [ ]:
# Create a network
network = ScalarNetwork(input_dim=2, hidden_dim=10, output_dim=1)

# Define an optimizer
optimizer = SGDOptimizer(parameters=network.parameters(), lr=0.3)

# Data
X = [list(x) for x in dataset.xs]
y_true = dataset.ys

In [ ]:
# Training loop
n_epochs = 150
losses = []

for epoch in range(n_epochs): 
    
    # Zero all grads
    optimizer.zero_grad()
    
    # Forward
    y_hat = network.forward(X)

    # Convert to binary class probabilties
    y_hat = [[scalar.sigmoid() for scalar in row] for row in y_hat]
    y_hat = list(itertools.chain.from_iterable(y_hat))
    
    # Compute a loss
    loss_per_epoch = sl.binary_cross_entropy(y_true, y_hat)
    loss_per_epoch.backward()
    
    optimizer.step()
    
    # Record
    losses.append(loss_per_epoch.data)
    if (epoch % 10 == 0) or (epoch == (n_epochs - 1)):
        print(f"epoch {epoch}: loss = {loss_per_epoch.data}")

In [ ]:
# Metrics
y_true = [Scalar(y_t) for y_t in y_true]
y_hat = [[scalar.sigmoid() for scalar in row] for row in network.forward(X)]
y_hat = list(itertools.chain.from_iterable(y_hat))

# Convert to classes using 0.5 threshold
y_hat_classes = [Scalar(1.0) if proba.data >= 0.5 else Scalar(0.0) for proba in y_hat]

{
    f.__name__: f(y_true=y_true, y_hat=y_hat_classes)
    for f in [sm.accuracy, sm.precision, sm.recall]
}

In [ ]:
# ROC Curve
tpr, fpr, thresholds = sm.roc_curve(y_true, y_hat, bucket_size=0.001)
tpr, fpr = [s.data for s in tpr], [s.data for s in fpr]

fig, ax = plt.subplots(1, 1, dpi=110)
ax.plot(fpr, tpr, "-o", c="tab:blue")
ax.plot([0, 1], [0, 1], ls="--", c="tab:grey")
ax.set_xlabel("fpr")
ax.set_ylabel("tpr")
ax.set_title("ROC Curve")

In [ ]:
# Visualising predictions

# Plot original data
fig, ax = plt.subplots(1, 1, dpi=110)
ax.scatter(list(x1_pos), list(x2_pos), marker="x", c="tab:blue")
ax.scatter(list(x1_neg), list(x2_neg), marker="o", c="tab:red")

# Generate mesh for predictions
n_steps = 101
x1_positions = list(np.linspace(0, 1.0, n_steps))
x2_positions = list(np.linspace(0, 1.0, n_steps))

# Generate predictions for each point on mesh
y_hat = np.zeros(shape=(n_steps, n_steps))
for i, x1 in enumerate(x1_positions):
    X = [[x1, x2] for x2 in x2_positions]
    y_hat_ = network.forward(X)
    y_hat_ = [[s.sigmoid().data for s in sample] for sample in y_hat_]
    y_hat[:, i] = np.array(y_hat_).flatten()


# Plot predictions as rectangles
for i, (x1_lower, x1_upper) in enumerate(zip(x1_positions, x1_positions[1:])):
    for j, (x2_lower, x2_upper) in enumerate(zip(x2_positions, x2_positions[1:])):
        rect = Rectangle(
            (x1_lower, x2_lower), 
            (x1_upper - x1_lower),
            (x2_upper - x2_lower), 
            color="tab:blue" if y_hat[i, j] >= 0.5 else "tab:red",
            alpha=(y_hat[i, j] - 0.5) if y_hat[i, j] >= 0.5 else (0.5 - y_hat[i, j]),
            lw=0.0
        )
        ax.add_patch(rect)

        
ax.set_xlabel("x1")
ax.set_ylabel("x2")
fig.tight_layout()

### Tensors

In [ ]:
# Create a network
network = TensorNetwork(input_dim=2, hidden_dim=10, output_dim=1)

# Define an optimizer
optimizer = SGDOptimizer(parameters=network.parameters(), lr=0.5)

# Data
X = tf.tensor([list(x) for x in dataset.xs])
y_true = tf.tensor(dataset.ys).view(N_SAMPLES, 1)

In [ ]:
# Training loop
n_epochs = 200
losses = []

for epoch in range(n_epochs): 
    
    # Zero all grads
    optimizer.zero_grad()
    
    # Forward
    y_hat = network.forward(X).sigmoid()

    # Compute a loss
    loss_per_epoch = tl.binary_cross_entropy(y_true, y_hat)
    loss_per_epoch.backward()
    
    optimizer.step()
    
    # Record
    losses.append(loss_per_epoch.item())
    if epoch % 10 == 0:
        print(f"epoch {epoch}: loss = {loss_per_epoch.item()}")

    if epoch == n_epochs - 1:
         print(f"epoch {epoch + 1}: loss = {loss_per_epoch.item()}")

In [ ]:
# Metrics
y_true = y_true.view(y_true.size)
y_hat = network.forward(X).sigmoid()
y_hat_classes = (y_hat >= 0.5).view(y_hat.size)

{
    f.__name__: f(y_true=y_true, y_hat=y_hat_classes)
    for f in [tm.accuracy, tm.precision, tm.recall]
}